<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Regression Models with Keras</font></h1>

## Introduction

As we discussed in the videos, despite the popularity of more powerful libraries such as PyToch and TensorFlow, they are not easy to use and have a steep learning curve. So, for people who are just starting to learn deep learning, there is no better library to use other than the Keras library. 

Keras is a high-level API for building deep learning models. It has gained favor for its ease of use and syntactic simplicity facilitating fast development. As you will see in this lab and the other labs in this course, building a very complex deep learning network can be achieved with Keras with only few lines of code. You will appreciate Keras even more, once you learn how to build deep models using PyTorch and TensorFlow in the other courses.

So, in this lab, you will learn how to use the Keras library to build a regression model.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Download and Clean Dataset</a>  
2. <a href="#item32">Import Keras</a>  
3. <a href="#item33">Build a Neural Network</a>  
4. <a href="#item34">Train and Test the Network</a>  

</font>
</div>

<a id="item31"></a>

## Download and Clean Dataset

Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [1]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

Let's download the data and read it into a <em>pandas</em> dataframe.

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>

Let's do a quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to *n_cols* since we will need this number when building our network.

In [12]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

<a id="item1"></a>

<a id='item32'></a>

## Import Keras

Recall from the videos that Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first and when you import the Keras library, it will be explicitly displayed what backend was used to install the Keras library. In CC Labs, we used TensorFlow as the backend to install Keras, so it should clearly print that when we import Keras.

#### Let's go ahead and import the Keras library

In [15]:
# Install Keras with it does not exist (Manuel hat das geschrieben)
!conda install keras --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - keras


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _tflow_select-2.2.0        |            eigen           3 KB
    absl-py-0.9.0              |           py36_0         166 KB
    astor-0.8.0                |           py36_0          45 KB
    ca-certificates-2020.1.1   |                0         165 KB
    certifi-2019.11.28         |           py36_1         157 KB
    conda-4.8.3                |           py36_0         3.1 MB
    gast-0.3.3                 |             py_0          14 KB
    grpcio-1.27.2              |   py36h351948d_0         1.3 MB
    keras-2.3.1                |                0           6 KB
    keras-applications-1.0.8   |             py_0          33 KB
    keras-base-2.3.1           |           py36_0       

In [16]:
import keras

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


As you can see, the TensorFlow backend was used to install the Keras library.

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [17]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>

## Build a Neural Network

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [18]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has two hidden layers, each of 50 hidden units.

<a id="item4"></a>

<a id='item34'></a>

## Train and Test the Network

Let's call the function now to create our model.

In [19]:
# build the model
model = regression_model()

Instructions for updating:
Colocations handled automatically by placer.


Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 100 epochs.

In [21]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=500, verbose=2)

Train on 721 samples, validate on 309 samples
Epoch 1/500
 - 0s - loss: 35.9077 - val_loss: 134.0145
Epoch 2/500
 - 0s - loss: 34.6278 - val_loss: 132.1973
Epoch 3/500
 - 0s - loss: 34.3083 - val_loss: 131.1432
Epoch 4/500
 - 0s - loss: 34.0857 - val_loss: 132.4807
Epoch 5/500
 - 0s - loss: 33.3702 - val_loss: 132.4018
Epoch 6/500
 - 0s - loss: 33.0001 - val_loss: 141.8390
Epoch 7/500
 - 0s - loss: 32.5367 - val_loss: 132.5128
Epoch 8/500
 - 0s - loss: 32.4680 - val_loss: 130.3126
Epoch 9/500
 - 0s - loss: 31.8369 - val_loss: 132.6939
Epoch 10/500
 - 0s - loss: 31.5850 - val_loss: 126.5248
Epoch 11/500
 - 0s - loss: 31.0174 - val_loss: 132.4397
Epoch 12/500
 - 0s - loss: 30.6723 - val_loss: 131.5235
Epoch 13/500
 - 0s - loss: 30.2008 - val_loss: 136.2449
Epoch 14/500
 - 0s - loss: 29.6798 - val_loss: 125.0347
Epoch 15/500
 - 0s - loss: 29.9133 - val_loss: 133.2505
Epoch 16/500
 - 0s - loss: 29.2734 - val_loss: 125.4135
Epoch 17/500
 - 0s - loss: 29.2606 - val_loss: 128.5523
Epoch 18/50

Epoch 146/500
 - 0s - loss: 15.0712 - val_loss: 121.5289
Epoch 147/500
 - 0s - loss: 14.8340 - val_loss: 122.1886
Epoch 148/500
 - 0s - loss: 14.7539 - val_loss: 121.2214
Epoch 149/500
 - 0s - loss: 14.7052 - val_loss: 119.4968
Epoch 150/500
 - 0s - loss: 14.9581 - val_loss: 123.5648
Epoch 151/500
 - 0s - loss: 14.6660 - val_loss: 124.8290
Epoch 152/500
 - 0s - loss: 15.0611 - val_loss: 122.2618
Epoch 153/500
 - 0s - loss: 14.6583 - val_loss: 119.2122
Epoch 154/500
 - 0s - loss: 14.6958 - val_loss: 119.7382
Epoch 155/500
 - 0s - loss: 14.8160 - val_loss: 117.0977
Epoch 156/500
 - 0s - loss: 14.4605 - val_loss: 117.7558
Epoch 157/500
 - 0s - loss: 14.9626 - val_loss: 116.6417
Epoch 158/500
 - 0s - loss: 14.6690 - val_loss: 121.7668
Epoch 159/500
 - 0s - loss: 14.4443 - val_loss: 118.3386
Epoch 160/500
 - 0s - loss: 14.4020 - val_loss: 123.8964
Epoch 161/500
 - 0s - loss: 14.5131 - val_loss: 122.7244
Epoch 162/500
 - 0s - loss: 14.3866 - val_loss: 122.8214
Epoch 163/500
 - 0s - loss: 14.

Epoch 290/500
 - 0s - loss: 11.9091 - val_loss: 114.2357
Epoch 291/500
 - 0s - loss: 11.5373 - val_loss: 129.9014
Epoch 292/500
 - 0s - loss: 11.8781 - val_loss: 123.9949
Epoch 293/500
 - 0s - loss: 11.5746 - val_loss: 117.4218
Epoch 294/500
 - 0s - loss: 11.5487 - val_loss: 116.9637
Epoch 295/500
 - 0s - loss: 11.2510 - val_loss: 120.9178
Epoch 296/500
 - 0s - loss: 12.1330 - val_loss: 127.0455
Epoch 297/500
 - 0s - loss: 11.5803 - val_loss: 121.4196
Epoch 298/500
 - 0s - loss: 11.7072 - val_loss: 118.5108
Epoch 299/500
 - 0s - loss: 11.4054 - val_loss: 119.8160
Epoch 300/500
 - 0s - loss: 11.3815 - val_loss: 122.4669
Epoch 301/500
 - 0s - loss: 11.7116 - val_loss: 121.4106
Epoch 302/500
 - 0s - loss: 11.2287 - val_loss: 122.4469
Epoch 303/500
 - 0s - loss: 11.1703 - val_loss: 114.9770
Epoch 304/500
 - 0s - loss: 11.1741 - val_loss: 123.6104
Epoch 305/500
 - 0s - loss: 11.7572 - val_loss: 118.4002
Epoch 306/500
 - 0s - loss: 11.1469 - val_loss: 119.8488
Epoch 307/500
 - 0s - loss: 11.

Epoch 434/500
 - 0s - loss: 10.2873 - val_loss: 121.0074
Epoch 435/500
 - 0s - loss: 10.1207 - val_loss: 118.7031
Epoch 436/500
 - 0s - loss: 9.9935 - val_loss: 128.9331
Epoch 437/500
 - 0s - loss: 10.1366 - val_loss: 126.2453
Epoch 438/500
 - 0s - loss: 9.9554 - val_loss: 118.5013
Epoch 439/500
 - 0s - loss: 10.1263 - val_loss: 125.8971
Epoch 440/500
 - 0s - loss: 9.7109 - val_loss: 120.7276
Epoch 441/500
 - 0s - loss: 9.7189 - val_loss: 129.2049
Epoch 442/500
 - 0s - loss: 9.8275 - val_loss: 127.0054
Epoch 443/500
 - 0s - loss: 9.7730 - val_loss: 118.8214
Epoch 444/500
 - 0s - loss: 10.1002 - val_loss: 122.4732
Epoch 445/500
 - 0s - loss: 9.9025 - val_loss: 130.9414
Epoch 446/500
 - 0s - loss: 10.0468 - val_loss: 126.2981
Epoch 447/500
 - 0s - loss: 10.7174 - val_loss: 113.6622
Epoch 448/500
 - 0s - loss: 9.9434 - val_loss: 126.7007
Epoch 449/500
 - 0s - loss: 9.7403 - val_loss: 126.4297
Epoch 450/500
 - 0s - loss: 9.8090 - val_loss: 119.3112
Epoch 451/500
 - 0s - loss: 9.8320 - val_

<strong>You can refer to this [link](https://keras.io/models/sequential/) to learn about other functions that you can use for prediction or evaluation.</strong>

Feel free to vary the following and note what impact each change has on the model's performance:

1. Increase or decreate number of neurons in hidden layers
2. Add more hidden layers
3. Increase number of epochs

### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!

This notebook is part of a course on **Coursera** called *Introduction to Deep Learning & Neural Networks with Keras*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0101EN_Coursera_Week3_LAB1).

<hr>

Copyright &copy; 2019 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).